# Early Fusion

This script takes all frames at once and passes them through a cnn.

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import time
import numpy as np

## 1. Load Data

To begin, data for a middle frames model must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [2]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/middle_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [3]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [4]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)
training_videos = np.swapaxes(training_videos, 3, 4)

In [6]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [7]:
#training_datafile = "E:/31-12-2020/prepared-data/middle_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/middle_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [8]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [9]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#training_videos = np.swapaxes(training_videos, 3, 4)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)
#validation_videos = np.swapaxes(validation_videos, 3, 4)

In [10]:
# Scale the data
#training_videos[:, :, :, :, 0] = (training_videos[:, :, :, :, 0] - np.min(training_videos[:, :, :, :, 0])) / (np.max(training_videos[:, :, :, :, 0]) - np.min(training_videos[:, :, :, :, 0]))
#training_videos[:, :, :, :, 1] = (training_videos[:, :, :, :, 1] - np.min(training_videos[:, :, :, :, 1])) / (np.max(training_videos[:, :, :, :, 1]) - np.min(training_videos[:, :, :, :, 1]))
#
#validation_videos[:, :, :, :, 0] = (validation_videos[:, :, :, :, 0] - np.min(validation_videos[:, :, :, :, 0])) / (np.max(validation_videos[:, :, :, :, 0]) - np.min(validation_videos[:, :, :, :, 0]))
#validation_videos[:, :, :, :, 1] = (validation_videos[:, :, :, :, 1] - np.min(validation_videos[:, :, :, :, 1])) / (np.max(validation_videos[:, :, :, :, 1]) - np.min(validation_videos[:, :, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [34]:
def model_generator(input_shape=(28, 2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv3D(32, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Second layer
    model.add(Conv3D(32, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Third layer
    model.add(Conv3D(32, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [35]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.3,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [36]:
# Flexible parameters
learning_rate = 0.1

In [37]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "EF_8_8_0__13__22_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Train on 191 samples, validate on 83 samples
Epoch 1/50
191/191 [==============================] - 162s 849ms/sample - loss: 11411.0593 - val_loss: 10635.6490
Epoch 2/50
191/191 [==============================] - 151s 792ms/sample - loss: 9990.9357 - val_loss: 10635.6490
Epoch 3/50
140/191 [====================>.........] - ETA: 47s - loss: 10349.9944

KeyboardInterrupt: 